# 6 Stochastische Optimierung und Genetische Algorithmen
In diesem Notebook wollen wir uns letztlich mit <a href="https://de.wikipedia.org/wiki/Stochastik" target="_blank">stochastischer</a> <a href="https://de.wikipedia.org/wiki/Optimierung_(Mathematik)" target="_blank">Optimierung</a> und <a href="https://de.wikipedia.org/wiki/Evolution%C3%A4rer_Algorithmus#Genetische_Algorithmen_(GA)" target="_blank">genetischen Algorithmen</a> befassen. Wir tun das zwar hauptsächlich auf einem pragmatischen Niveau, aber auch sodass wir in der kurzen Zeit bereits etwas damit anfangen können. 

Zu diesem Zweck brauchen wir kurz ein paar Vorüberlegungen zu den Begriffen <a href="https://de.wikipedia.org/wiki/Zufallszahl" target="_blank">Zufallszahlen</a> und <a href="https://de.wikipedia.org/wiki/Stichprobe" target="_blank">Sampling (Stichproben)</a>, und zwar hauptsächlich, wie wir diese in <a href="https://www.python.org/" target="_blank">Python</a> bekommen und woher.

## 6.1 Zufallszahlen
Echte Zufallszahlen sind schwer zu erzeugen. Daher bedienen sich Computer-Nutzer sogenannter Pseudo-Zufallszahlen. Damit ist gemeint, dass man einen <a href="https://de.wikipedia.org/wiki/Zufallszahlengenerator" target="_blank">(Pseudo-Zufallszahlen-)Generator</a> hat, der eine bestimmte Methode verfolgt, um die gewünschte Anzahl von Zahlen so zu erzeugen, dass sie möglichst gut der gewünschten <a href="https://de.wikipedia.org/wiki/Wahrscheinlichkeitsma%C3%9F" target="_blank">Wahrscheinlichkeitsverteilung</a> entsprechen. Sehen wir uns das gleich einmal anhand einiger Beispiele an.

Zunächst die Imports von heute:

In [ ]:
%matplotlib inline

import numpy as np

# Importiere Statistische Package aus SciPy
import scipy.stats as scs

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from tqdm import tqdm


<a href="https://numpy.org/" target="_blank">NumPy</a> hat ein Modul namens _random_, in dem man die wichtigsten Dinge finden kann, z.B.:

In [ ]:
# Zufallszahlen zwischen 0 und 1, gleichverteilt

# Fixiere den Seed, um reproduzierbare Ergebnisse zu erhalten:
#np.random.seed(1234)

# Generiere ein Feld von Zufallszahlen
np.random.random(size=(4, 5))

In [ ]:
# Zufallszahlen zwischen 1 und 2
a = 1
b = 2
(b - a) * np.random.random(size=(4, 5)) + a

In [ ]:
# Zufallszahlen aus der Normalverteilung
normal_values_x = np.random.normal(0., 1., size=(1000))
normal_values_y = np.random.normal(0., 1., size=(1000))

fig = plt.figure()

plt.scatter(normal_values_x, normal_values_y, s=0.1)

plt.show()

Zur Erinnerung: Wir können jederzeit die Eigenschaften einer so erzeugten Verteilung überprüfen, z.B. den <a href="https://de.wikipedia.org/wiki/Mittelwert" target="_blank">Mittelwert</a> und die <a href="https://de.wikipedia.org/wiki/Varianz" target="_blank">Standardabweichung</a>, aber auch höhere Momente wie <a href="https://de.wikipedia.org/wiki/Schiefe_(Statistik)" target="_blank">Skewness</a> oder <a href="https://de.wikipedia.org/wiki/W%C3%B6lbung_(Statistik)" target="_blank">NumPy</a>Kurtosis:

In [ ]:
# checke Mittelwert
print("Mittelert:", np.mean(normal_values_x))

# checke Standardabweichung
print("Standardabweichung:", np.std(normal_values_x))

# checke Skewness (Asymmetrie)
print("Skewness:", scs.skew(normal_values_x))

# checke Kurtosis ("Dicke der Extrembereiche im Vergleich zu Normal")
print("Kurtosis:", scs.kurtosis(normal_values_x))


## 6.2 Sampling
Mit diesen ersten Schritten haben wir ein Gefühl für das Arbeiten mit Zufallszahlen in Python bekommen. Der nächste Schritt ist, aus einer Vorhandenen Menge von Daten eine Stichprobe zu ziehen. Man nennt diesen Schritt auch einfach _Sampling_ (vom englischen Begriff her). 

Zunächst sollte ich erwähnen, dass wir bereits im vorigen Abschnitt "gesampelt" haben, denn die entsprechenden _NumPy_-Funktionen ziehen ja auch eine Stichprobe aus der gewünschten Verteilung. Wenn es also nur darum geht, Werte aus einer der Standard-Verteilungen zu bekommen, dann benutzt man einfach den entsprechenden Befehl.

Wenn wir allerdings Daten vorgegeben haben und aus diesen unsere Stichprobe ziehen wollen, dann müssen wir das den Computer aus einer bestimmten Menge (einem Array) zufällig die gewünschte Anzahl von Werten ziehen lassen. In _NumPy_ gibt es auch dafür einen Befehl.

In [ ]:
# Erzeuge ein einfaches Array von Daten
all_data = np.arange(1, 20)

# hole mir eine zufällige 3x5 Matrix von Werten aus diesem Array von 1 bis 20
np.random.choice(all_data, size=(3, 5))


In [ ]:
# hier kommen einige Zahlen mehrfach vor. Wenn man das nicht möchte:
np.random.choice(all_data, size=(17), replace=False)

In [ ]:
# und schließlich kann man noch ein Array mit Wahrscheinlichkeiten
# übergeben, nach der die Elemente bestimmt werden sollen

# Beispiel: Ein Würfel mit den Augenzahlen 1, 1, 1, 2, 2, 3
# zunächst die Möglichkeiten für die Augen
die_faces = np.array([1, 2, 3])

# Dann die relativen Häufigkeiten
die_probs_raw = np.array([3, 2, 1])

# das Argument p muss jedoch auf 1 normiert sein
die_probs = die_probs_raw/np.sum(die_probs_raw)

print("Wahrscheinlichkeiten für 1, 2, 3:", die_probs, "\n")

# Erzeuge nun einige Würfe mit diesem Würfel
n_rolls = 300
die_rolls = np.random.choice(die_faces, size=n_rolls, p=die_probs)

print("Ein paar Würfe:", die_rolls)

In [ ]:
# Passt das zusammen?

# Was sollten die Ergebnisse sein?
print("Theoretische Anzahl:", n_rolls * die_probs)

fig = plt.figure()

plt.hist(die_rolls, bins=[1,2,3,4])

plt.show()

## 6.3 Stochastische Optimierung
Mit diesen Werkzeugen können wir uns an die stochastische Optimierung wagen. Was bedeutet das eigentlich? Wir hatten in der vorigen Einheit mit Optimierung zu tun, ganz im Allgemeinen. Dabei sucht man grundsätzlich das Optimum einer Funktion von meist mehreren Variablen.

Die Methoden, die wir dabei besprochen haben waren zunächst einmal "brute force", also alle Möglichkeiten durchzugehen und die beste über einen globalen Vergleich zu identifizieren. Danach haben wir aber auch noch mit "Gradient Descent" experimentiert, bei dem man die Ableitung der Funktion nutzt, um schrittweise an den tiefsten Punkt zu kommen. 

Ein konkretes Problem bei Gradient Descent z.B. ist, dass man in einem lokalen Minimum "steckenbleiben" kann, wenn der Weg dort hinein führt und die Schrittweite zu klein ist, um wieder herauszukommen, obwohl das globale Minimum ein anderes ist. Man kann sich das so vorstellen wie eine riesige Berg- und Tal-Landschaft mit vielen Tälern, Mulden, Gipfeln, Bergrücken, Löchern, etc., und irgendeins davon ist der niedrigste Punkt. Insbesondere wenn diese Landschaft in hochdimensionalen Räumen betrachtet wird, leuchtet ein, dass man es mit Gradient Descent vielleicht schwer haben könnte.

Das hat vor allem auch damit zu tun, dass man diese Funktions-"Landschaft" gar nicht wirklich kennt, weil man entweder keine Ahnung hat, wie die Funktion überhaupt aussieht, oder weil es sehr teuer ist, die Funktion zu berechnen. Vor allem in so einem Fall hilft es, wenn man sich nicht die gesamte Landschaft ansehen muss, sondern Schritt für Schritt mit sehr wenigen Werten zu einer Lösung kommen kann.

Eine mögliche Alternative hier ist, einfach viele zufällige Werte aus dem Wertebereich zu nehmen (aber längst nicht alle, also ein Sample), und für diese Werte den Wert der Funktion zu bestimmen. Der kleinste davon ist dann eine Näherung für das Minimum. Im folgenden Beispiel ist aus Gründen der Anschaulichkeit die Funktion bekannt, sodass wir sie auch plotten können und uns den Algorithmus etwas ansehen können. Behalten Sie aber bitte im Kopf, dass wir das im Normalfall nicht hätten, sondern einfach nur die Samples, für die wir die Funktion berechnen. Sehen wir uns gleich einmal an, ob sowas funktioniert:

In [ ]:
# definiere eine Funktion für die Suche nach deren Minimum
def a_landscape(x, y):
    # mehrere Täler und Berge, mit einer globalen Neigung
    our_function = 4*np.sin(x) + 6*np.sin(y) - 0.5 * x - 0.2 * y

    return our_function

        
# plotten wir das mal

# definiere x-Werte
x_vals = np.linspace(-10, 10, 500)

# definiere y-Werte
y_vals = np.linspace(-10, 10, 500)

# erzeuge x-y-Grid für 3D Plots
X, Y = np.meshgrid(x_vals, y_vals)

Z = a_landscape(X, Y)

# neue Grafik
fig = plt.figure()

# 3D Achsen erzeugen
ax = fig.add_subplot(1,1,1, projection='3d')

# erzeuge 3D-Oberflächenplot
ax.plot_surface(X, Y, Z, cmap="magma")

plt.show()

In [ ]:
# für Reproduzierbarkeit die nächste Zeile verwenden
# np.random.seed(12345)

sample_size = 100

# nun wählen wir einen Satz Werte zufällig aus unserem Bereich:
# Wertebereich für x und y
a = -10
b =  10


def evaluate_a_sample(size=sample_size, verbose=False):
    # ziehe eine Stichprobe von 2xSamplesize (für x und y)
    test_sample = (b - a) * np.random.random(size=(2, sample_size)) + a

    # Werte die Funktion an diesen Punkten aus
    sample_values = a_landscape(*test_sample)  

    if verbose: print("Alle Funktionswerte des Samples:\n", sample_values)

    the_minimum = np.min(sample_values)

    if verbose: print("Der kleinste Wert im Sample:", the_minimum)
    
    return the_minimum

evaluate_a_sample(100, verbose=True)

In [ ]:
# Anzahl der Samples
n_samples = 200

# Eine Liste mit Resultaten anlegen
min_list = []

# Mache die Auswertung für einige Samples
for i in range(n_samples):
    min_list.append(evaluate_a_sample(100))
    

print("Liste der Minima:", min_list, "\n")

print("Minimum der Minima:", np.min(min_list))
    
fig = plt.figure()

plt.hist(min_list)

plt.show()

Letzten Endes wird man die Sache aber etwas cleverer angehen als wir hier in diesem Beispiel und z.B. bei jedem neuen Sample die Position des Minimums (oder der 10 kleinsten Werte) aus dem vergangenen Sample als Basis dafür hernehmen, wo die Punkte des neuen Samples konzentriert werden sollten. Dadurch konzentriert man die Suche auf interessante Bereiche.

Diese Taktik geht bereits in Richtung des nächsten Themas und ist daher eine gute Überleitung, nämlich:


## 6.4 Genetische Algorithmen
Ein genetischer Algorithmus ist eine Taktik zur stochastischen Optimierung, d.h., zum Finden der optimalen Parameter für eine Funktion, sodass diese minimal oder maximal wird. Die Basis für die "Genetik" bei der Optimierung ist eine Art <a href="https://de.wikipedia.org/wiki/Code" target="_blank">Codierung</a> des Inputs, z.B. ein Vektor von Zahlen. Aus diesem Vektor wird dann der Wert der zu optimierenden Funktion berechnet, die in diesem Zusammenhang "Fitnessfunktion" heißt.

Bei der Optimierung selbst geht man über mehere Schritte, die mit "Generationen" verglichen werden, weil sie mehrere "Individuen" dieser Vektoren enthalten. Von einer Generation zur nächsten gibt eine Auswahl der "fittesten" Individuen, genannt "Eltern", ihre "Erbinformation" weiter. Dadurch entstehen im Laufe der Generationen immer fittere Individuen, d.h. wir nähern uns dem Optimum der Fitnessfunktion. 

## 6.5 Beispiel: Das binäre Rucksackproblem

Warum ist das interessant? Nehmen wir z.B. ein kombinatorisches Optimierungsproblem etwas genauer unter die Lupe, das als <a href="https://de.wikipedia.org/wiki/Rucksackproblem" target="_blank">"binäres Rucksackproblem"</a> bezeichnet wird. Denken Sie dabei etwa an einen Fahrradboten. Dieser hat einen Rucksack für den Transport von Gütern, die er ausliefert. Jedes Ding, das er liefern kann, hat dabei ein Gewicht und einen Preis. Der Bote packt für eine Fahrt den Rucksack voll mit Dingen, sodass der gesamte Preis möglichst hoch wird, das gesamte Gewicht jedoch die Kapazität des Rucksacks nicht überschreitet. Dabei darf er jedes Ding entweder nicht mitnehmen oder genau einmal mitnehmen (daher binär).

Dieses Problem ist, wie gesagt, <a href="https://de.wikipedia.org/wiki/Kombinatorik" target="_blank">kombinatorisch</a>. Man kann, um es zu lösen, im Prinzip auch per "brute force" alle Möglichkeiten durchgehen und dafür das Maximum der aufsummierten Preise finden. Das wird allerdings sehr schnell sehr langwierig. Bei der Abkürzung der Zeit für die Lösung hilft der genetische Algorithmus.

Sehen wir uns das nun im Detail an.

In [ ]:
# Tabelle der Anzahl der Möglichkeiten, Dinge 
# in den Rucksack zu packen, wenn es _N_ zur Auswahl gibt
for n in np.arange(1, 62, 5):
    # für jedes Ding gibt es 2 Möglichkeiten (mit, nicht mit)
    # daher insgesamt 2**n
    print(n, "->", 2**n)

In [ ]:
# wie lange dauert sowas ca.? 
# nehmen wir eine Mikrosekunde für eine Kombination an
2**61 * 1.e-6 / 3600 / 24 / 365   # hier sollten Jahre herauskommen

Nun ist soweit klar, dass es mit der Zeit relativ schnell knapp wird. Wie löst man so ein Problem aber jetzt konkret mit einem genetischen Algorithmus? Wir brauchen folgendes.

* Ein Encoding der "genetischen Information" für jedes Individuum einer Generation/Population (also hier für jede Kombination von Dingen, die eingepackt werden)
* Einen Mechanismus, der uns die erste Generation von Individuen erzeugt
* Einen Mechanismus (oder mehrere), der den genetischen Code von Individuen von einer zur nächsten Generation verändern kann
* Eine Vorschrift, mit der die Fitness jedes Individuums berechnet werden kann
* Eine Vorschrift dafür, welche und wie viele der fittesten Individuen einer Generation zu Eltern für die nächste Generation werden

Wir brauchen natürlich auch die Basis für dieses Problem, nämlich die Liste mit den Dingen, ihren Gewichten und Preisen, sowie das Gewichtslimit für den Rucksack. Aber dann kann es schon losgehen. Also legen wir los.

In [ ]:
# Setze das Limit für den Rucksack
weight_limit = 15

# Setze N
num_items = 20

# Erzeugen der Liste der Dinge mit Gewichten und Preisen
item_weights = np.round((3.5 - 0.2) * np.random.random(size=num_items) + 0.2, 2)
item_prices  = np.round((30. - 0.1) * np.random.random(size=num_items) + 0.1, 2)

In [ ]:
# was ist zum Beispiel mit Ding nummer 5?
(item_weights[4], item_prices[4])

In [ ]:
# Das Encoding für eine Variante des Einpackens 
# ist ein Vektor mit _N_ Zahlen, die entweder 1 oder 0 sein können
test_encoding = np.random.choice([0, 1], size=num_items, replace=True)
test_encoding


In [ ]:
# Damit kann man leicht das Gewicht für eine Variante ausrechnen
the_weight = np.sum(item_weights * test_encoding)
the_weight

In [ ]:
# Ebenso funktioniert der Gesamtpreis
the_price = np.sum(item_prices * test_encoding)
the_price

In [ ]:
# Testen wir nun einmal kurz die brute-force-Lösung
# D.h. wir gehen durch alle Möglichkeiten und suchen die beste raus

# Zunächst definieren wir aber die Fitness-Funktion:
# wir schreiben diese Funktion so, dass sie mehrere Encodings
# gleichzeitig abarbeiten kann
def fitness(encodings):
    # berechne Gewicht und Preis
    the_weights = np.sum(item_weights * encodings, axis=1)
    the_prices = np.sum(item_prices * encodings, axis=1)
    
    # testen, ob das Gewicht in den Rucksack passt
    the_prices = np.where(the_weights <= weight_limit * np.ones(len(encodings)), 
                          # ja, passt, die Fitness ist der Preis
                          the_prices,  
                          # nein, passt nicht, die Fitness wird auf 0 gesetzt
                          np.zeros(len(encodings))
                         )
    
    return the_prices, the_weights


In [ ]:
# Nun zum Ausprobieren aller Möglichkeiten:

# setze Wert für besten Preis an
best_price = 0.

# setze Wert für bestes Encoding an
best_encoding = np.zeros(num_items)

# setze Wert für Gewicht zum besten Preis an
best_weight = 0.

# loop über alle natürlichen Zahlen bis 2**N
for counter in tqdm(np.arange(0, 2**num_items, 1)):
    
    # generiere binäre Darstellung des Zählers (als String) und ergänze führende Nullen
    counter_binary = np.binary_repr(counter).zfill(num_items)
    
    # verwandle den String in ein numpy-Array mit 0en und 1en
    current_encoding = np.array([int(a_letter) for a_letter in counter_binary])
        
    # berechne den Preis (inklusive Gewichtscheck) über die Fitness-Funktion
    the_price, the_weight = fitness([current_encoding])
    
    # Abfrage, ob wir eine bessere Lösung gefunden haben als bisher
    if the_price > best_price:
        
        # ersetze Bestwerte für Preis, Encoding und Gewicht
        best_price    = the_price
        best_encoding = current_encoding
        best_weight   = the_weight
        
        
print("The best price is", best_price)
print("at a weight of", best_weight)
print("The best encoding is\n", best_encoding)
    

In [ ]:
# Nun zum genetischen Algorithmus

# definiere Anzahl der Eltern
num_parents = 20

# definiere Anzahl Kinder pro Elternpaar
num_c_p_p = 3

# definiere Anzahl der Individuen in einer Generation
# entspricht dem Quadrat der Anzahl der Eltern 
# (wegen der Möglichkeiten von Crossovers)
generation_size = num_c_p_p * num_parents**2

# definiere maximale Anzahl der Generationen
max_generations = 10

# definiere Crossover von zwei Encodings, d.h. rekombiniere die beiden
# an einer zufällig gewählten Stelle
def crossover(encoding_1, encoding_2):
    
    # wähle eine zufällige Stelle aus der Länge der Encodings
    cut_position = np.random.choice(range(num_items))
    
    # rekombiniere die beiden Arrays zu einem mit Schnitt an dieser Stelle
    new_encoding = np.hstack((encoding_1[:cut_position], encoding_2[cut_position:]))
    
    return new_encoding

# erzeuge Generation 0
next_generation = np.random.choice([0, 1], size=(generation_size, num_items), replace=True)

# Loop über die Generationen
for count_generations in range(max_generations):
    
    # print(next_generation)
    
    # bestimme die Fitness aller Individuen in dieser Generation
    the_prices, the_weights = fitness(next_generation)
    
    # print(the_prices)
    
    # bestimme Reihenfolge der Indizes nach Fitness der Individuen
    the_ranking = np.argsort(the_prices)[::-1]
    
    # print(the_ranking)
        
    # suche die nächsten Eltern aus den besten aus
    the_parents = (next_generation[the_ranking])[:num_parents]
    
    # baue die nachfolgende Generation aus den Eltern plus 
    # num_parents x num_parents "Kindern" zusammen
    the_children = []
    
    # erzeuge Kinder durch Crossover, d.h. Kombination der genetischen
    # Codes der beiden Eltern an einer bestimmten Stelle
    # Erster Loop über alle Eltern
    for parent_1 in the_parents:
        
        # Zweiter Loop über alle Eltern
        for parent_2 in the_parents:
            
            # Zusätzlicher Loop über die Anzahl der Kinder pro Elternpaar
            for child_counter in range(num_c_p_p):
                
                # hänge alle möglichen Crossovers in eine Liste zusammen
                the_children.append(crossover(parent_1, parent_2))
    
    # mache aus der Liste ein Array
    next_generation = np.array(the_children)
    
    # hier sind auch die Eltern wieder dabei, weil ein Crossover eines Encodings
    # mit sich selbst das gleiche Encoding nochmals erzeugt
    
    
    # print("parents", the_parents)
    # print("children", the_children)
    # print("new gen.", next_generation)
    
    
    # Output für diese Generation
    print("Gen.:", count_generations, "Beste Fitness:", np.round(np.max(the_prices), 2),
          "mit Gewicht ", np.round(the_weights[the_ranking[0]], 2), "bestes Encoding:", the_parents[0])
        


## 6.6 Übungsaufgabe: Pushen Sie das Rucksackproblem an die Grenzen
Nehmen Sie den Code von eben und experimentieren Sie damit. Sie könnten z.B. damit folgendes tun:

* Verändern Sie die Anzahl der Eltern (und damit auch die Generationsgröße) und prüfen Sie, ob die Lösung dadurch näher an die brute-force-Lösung herankommt
* Verändern Sie die Anzahl der Kinder pro Elternpaar und überprüfen Sie den Effekt
* Verändern Sie das Limit für den Rucksack
* Verändern Sie die Limits für die Preise und Gewichte
* Finden Sie heraus, wie weit Sie $N$ auf Ihrem Rechner ohne Probleme nach oben schrauben können
* Implementieren Sie ein Timing für Teile eines Runs und den gesamten Run
* Visualisieren Sie die Entwicklung des Timings mit $N$
